In [309]:
import pandas as pd
import numpy as np
import re

In [266]:
notes = pd.read_csv('NOTEEVENTS.csv')

C:\Users\Jason\AppData\Local\Temp\ipykernel_21136\2856085482.py:1: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  notes = pd.read_csv('NOTEEVENTS.csv')


In [269]:
rel_cats = ['SUBJECT_ID','HADM_ID','CHARTDATE','CHARTTIME','CATEGORY','TEXT']
notes = notes[rel_cats]
# Why have notes if they have no text
notes.dropna(subset=['TEXT'], inplace=True)
# Make it friendly for datetime comparisons later on
notes.CHARTDATE = notes.CHARTDATE.astype('datetime64[ns]')
notes.CHARTTIME = notes.CHARTTIME.astype('datetime64[ns]')

In [270]:
notes.set_index(['SUBJECT_ID','HADM_ID'], inplace=True)

In [271]:
notes.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 2083180 entries, (22532, 167853.0) to (31097, 115637.0)
Data columns (total 4 columns):
 #   Column     Dtype         
---  ------     -----         
 0   CHARTDATE  datetime64[ns]
 1   CHARTTIME  datetime64[ns]
 2   CATEGORY   object        
 3   TEXT       object        
dtypes: datetime64[ns](2), object(2)
memory usage: 83.3+ MB


In [272]:
notes.head()

CHARTDATE CHARTTIME           CATEGORY  \
SUBJECT_ID HADM_ID                                            
22532      167853.0 2151-08-04       NaT  Discharge summary   
13702      107527.0 2118-06-14       NaT  Discharge summary   
           167118.0 2119-05-25       NaT  Discharge summary   
           196489.0 2124-08-18       NaT  Discharge summary   
26880      135453.0 2162-03-25       NaT  Discharge summary   

                                                                  TEXT  
SUBJECT_ID HADM_ID                                                      
22532      167853.0  Admission Date:  [**2151-7-16**]       Dischar...  
13702      107527.0  Admission Date:  [**2118-6-2**]       Discharg...  
           167118.0  Admission Date:  [**2119-5-4**]              D...  
           196489.0  Admission Date:  [**2124-7-21**]              ...  
26880      135453.0  Admission Date:  [**2162-3-3**]              D...

### Clean up and trim categories

Clean up category names

In [273]:
notes.loc[notes.CATEGORY == 'Discharge summary', 'CATEGORY'] = 'Discharge'
notes.loc[notes.CATEGORY == 'Physician ', 'CATEGORY'] = 'Physician'
notes.loc[notes.CATEGORY == 'Respiratory ', 'CATEGORY'] = 'Respiratory'

Select common categories

In [274]:
relevant_cats = ["Discharge","Nursing/other","Radiology","Nursing","ECG","Physician","Echo","Respiratory","Nutrition","General"]
notes = notes.loc[notes.CATEGORY.map(lambda x: x in relevant_cats)]

## Add ICUSTAY_ID to notes

In [275]:
# Load in dataframe
icustays = pd.read_csv('ICUSTAYS.csv')
# Convert from 'object' to 'datetime'
icustays.INTIME = icustays.INTIME.astype('datetime64[ns]')
icustays.OUTTIME = icustays.OUTTIME.astype('datetime64[ns]')
# Create date columns in case row in question doesn't have a 'CHARTTIME' for comparison
icustays['INDATE'] = icustays.INTIME.dt.date.astype('datetime64[ns]')
icustays['OUTDATE'] = icustays.OUTTIME.dt.date.astype('datetime64[ns]')
# Select only the first ICU stay for a patient across all admissions
icustays = icustays.sort_values(by='INTIME').groupby(['SUBJECT_ID']).first()
# Select the minimal information needed to match ICU stays to clinical notes
icustays = icustays[["HADM_ID","ICUSTAY_ID","INTIME","INDATE","OUTTIME","OUTDATE"]]
# Impossible to verify notes belong to an ICU stay without a time
icustays.dropna(subset=['INTIME','OUTTIME'], inplace=True)
# Each index is a single patient's first ICU visit (we removed multiple ICU visits)
icustays.reset_index(inplace=True)
icustays.set_index(['SUBJECT_ID','HADM_ID','ICUSTAY_ID'], inplace=True)

In [276]:
icustays.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 46467 entries, (2, 163353, 243653) to (99999, 113369, 246512)
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   INTIME   46467 non-null  datetime64[ns]
 1   INDATE   46467 non-null  datetime64[ns]
 2   OUTTIME  46467 non-null  datetime64[ns]
 3   OUTDATE  46467 non-null  datetime64[ns]
dtypes: datetime64[ns](4)
memory usage: 6.0 MB


In [277]:
icustays.head()

,,,INTIME,INDATE,OUTTIME,OUTDATE
SUBJECT_ID,HADM_ID,ICUSTAY_ID,,,,
2,163353,243653,2138-07-17 21:20:07,2138-07-17,2138-07-17 23:32:21,2138-07-17
3,145834,211552,2101-10-20 19:10:11,2101-10-20,2101-10-26 20:43:09,2101-10-26
4,185777,294638,2191-03-16 00:29:31,2191-03-16,2191-03-17 16:46:31,2191-03-17
5,178980,214757,2103-02-02 06:04:24,2103-02-02,2103-02-02 08:06:00,2103-02-02
6,107064,228232,2175-05-30 21:30:54,2175-05-30,2175-06-03 13:39:54,2175-06-03


### Join to `notes`

In [298]:
notes_icu = icustays.copy()
notes_icu = notes_icu.join(notes, how='inner')

### Remove mismatched notes

In [299]:
# Date comparison
date_val = (notes_icu.CHARTDATE >= notes_icu.INDATE) & (notes_icu.CHARTDATE <= notes_icu.OUTDATE)
# Time comparison
time_val = (notes_icu.CHARTTIME >= notes_icu.INTIME) & (notes_icu.CHARTTIME <= notes_icu.OUTTIME)
# Valid?
datetime_val = (notes_icu.CHARTTIME.isna() & date_val) | (~notes_icu.CHARTTIME.isna() & time_val) 
# Filter rows
notes_icu = notes_icu.loc[datetime_val]

In [300]:
notes_icu.head()

INTIME     INDATE  \
SUBJECT_ID HADM_ID ICUSTAY_ID                                  
2          163353  243653     2138-07-17 21:20:07 2138-07-17   
                   243653     2138-07-17 21:20:07 2138-07-17   
3          145834  211552     2101-10-20 19:10:11 2101-10-20   
                   211552     2101-10-20 19:10:11 2101-10-20   
                   211552     2101-10-20 19:10:11 2101-10-20   

                                          OUTTIME    OUTDATE  CHARTDATE  \
SUBJECT_ID HADM_ID ICUSTAY_ID                                             
2          163353  243653     2138-07-17 23:32:21 2138-07-17 2138-07-17   
                   243653     2138-07-17 23:32:21 2138-07-17 2138-07-17   
3          145834  211552     2101-10-26 20:43:09 2101-10-26 2101-10-21   
                   211552     2101-10-26 20:43:09 2101-10-26 2101-10-21   
                   211552     2101-10-26 20:43:09 2101-10-26 2101-10-21   

                                        CHARTTIME       CATEGORY  \
SUBJECT_ID HADM_ID ICUSTAY_ID                                      
2          163353  243653     2138-07-17 23:08:00  Nursing/other   
                   243653     2138-07-17 22:51:00  Nursing/other   
3          145834  211552                     NaT           Echo   
                   211552                     NaT           Echo   
                   211552                     NaT           Echo   

                                                                            TEXT  
SUBJECT_ID HADM_ID ICUSTAY_ID                                                     
2          163353  243653      Nursing Transfer note\n\n\nPt admitted to NICU...  
                   243653      Neonatology Attending Triage Note\n\nBaby [**N...  
3          145834  211552      PATIENT/TEST INFORMATION:\nIndication: S/P Car...  
                   211552      PATIENT/TEST INFORMATION:\nIndication: Pericar...  
                   211552      PATIENT/TEST INFORMATION:\nIndication: Left ve...

In [301]:
print(f"Total remaining clinical notes: {notes_icu.shape[0]}")

Total remaining clinical notes: 1226805


### Match Cohorts using `df_pats`

In [302]:
filename = 'all_hourly_data.h5'
df_pats = pd.read_hdf(filename, 'patients')[[]]

In [303]:
df_pats.head()

,,
subject_id,hadm_id,icustay_id
3,145834,211552
4,185777,294638
6,107064,228232
9,150750,220597
11,194540,229441


Match index names

In [304]:
notes_icu.index.names = ['subject_id','hadm_id','icustay_id']

In [305]:
notes_icu = df_pats.join(notes_icu, how='left')

In [306]:
notes_icu.head()

INTIME     INDATE  \
subject_id hadm_id icustay_id                                  
3          145834  211552     2101-10-20 19:10:11 2101-10-20   
                   211552     2101-10-20 19:10:11 2101-10-20   
                   211552     2101-10-20 19:10:11 2101-10-20   
                   211552     2101-10-20 19:10:11 2101-10-20   
                   211552     2101-10-20 19:10:11 2101-10-20   

                                          OUTTIME    OUTDATE  CHARTDATE  \
subject_id hadm_id icustay_id                                             
3          145834  211552     2101-10-26 20:43:09 2101-10-26 2101-10-21   
                   211552     2101-10-26 20:43:09 2101-10-26 2101-10-21   
                   211552     2101-10-26 20:43:09 2101-10-26 2101-10-21   
                   211552     2101-10-26 20:43:09 2101-10-26 2101-10-25   
                   211552     2101-10-26 20:43:09 2101-10-26 2101-10-26   

                              CHARTTIME CATEGORY  \
subject_id hadm_id icustay_id                      
3          145834  211552           NaT     Echo   
                   211552           NaT     Echo   
                   211552           NaT     Echo   
                   211552           NaT      ECG   
                   211552           NaT      ECG   

                                                                            TEXT  
subject_id hadm_id icustay_id                                                     
3          145834  211552      PATIENT/TEST INFORMATION:\nIndication: S/P Car...  
                   211552      PATIENT/TEST INFORMATION:\nIndication: Pericar...  
                   211552      PATIENT/TEST INFORMATION:\nIndication: Left ve...  
                   211552      Technically difficult study\nSinus rhythm with...  
                   211552      Sinus rhythm\nP-R interval increased\nLate R w...

### Preprocess text

remove newline characters

In [307]:
notes_icu_pre = notes_icu.copy()

In [346]:
def prepare_text(t):
    t_pre = re.sub('(\n+|\.|\?\!)', ' [SEP] ', t)
    t_pre = re.sub('\s+', ' ', t_pre)
    t_pre = re.sub('\s(\[SEP\]\s){2,}', ' [SEP] ', t_pre)
    t_pre = t_pre.strip()
    t_pre = re.sub('[^a-zA-Z0-9_\[\] ]', '', t_pre)
    
    return t_pre

notes_icu_pre.TEXT = notes_icu_pre.TEXT.map(lambda t: t if pd.isnull(t) else prepare_text(t))

In [347]:
sample_text = notes_icu_pre.TEXT.iloc[0]
sample_text

'PATIENTTEST INFORMATION [SEP] Indication SP Cardiac arrest [SEP] RO Pericardial effusion [SEP] Height in 69 [SEP] Weight lb 150 [SEP] BSA m2 1 [SEP] 83 m2 [SEP] BP mm Hg 8540 [SEP] Status Inpatient [SEP] DateTime [21011021] at 1030 [SEP] Test Portable TTEComplete [SEP] Doppler Complete pulse and color flow [SEP] Contrast None [SEP] Technical Quality Suboptimal [SEP] INTERPRETATION [SEP] Findings [SEP] LEFT ATRIUM The left atrium is normal in size [SEP] RIGHT ATRIUMINTERATRIAL SEPTUM The right atrium is normal in size [SEP] LEFT VENTRICLE Left ventricular wall thicknesses are normal [SEP] The left [SEP] ventricular cavity size is normal [SEP] There is severe regional left ventricular [SEP] systolic dysfunction [SEP] Overall left ventricular systolic function is severely [SEP] depressed [SEP] RIGHT VENTRICLE Right ventricular chamber size and free wall motion are [SEP] normal [SEP] The right ventricular wall thickness is normal [SEP] Right ventricular [SEP] chamber size is normal [SEP] 